In [ ]:
# Importing needed libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
from PIL import Image



In [ ]:
# connecting to drive
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/NTI project/garbage-dataset'
test_path = '/content/drive/MyDrive/NTI project/test_ds'

Mounted at /content/drive


In [ ]:
# checking the validity of data paths and the number of files in each class
# from genericpath import isdir
for classes in os.listdir(data_path):
  class_path = os.path.join(data_path, classes)
  # print(class_path)
  if os.path.isdir(class_path):
    files = os.listdir(class_path)
    print(f"{classes}: {len(files)} files")

battery: 851 files
shoes: 1777 files
cardboard: 1635 files
plastic: 1784 files
trash: 897 files
glass: 2761 files
clothes: 5195 files
paper: 1518 files
biological: 897 files
metal: 920 files


In [ ]:
# checking for non-image files

valid_extensions = ('.jpg','.png','.jepg')

for classes in os.listdir(data_path):
  class_path = os.path.join(data_path, classes)
  if os.path.isdir(class_path):
    for current_file in os.listdir(class_path):
      if not current_file.lower().endswith(valid_extensions):
        print(f"{current_file} is not an image")

# no non-image files detected

In [ ]:
#checking for corrupted files

from PIL import Image

corrupted_images = []

for classes in os.listdir(data_path):
  class_path = os.path.join(data_path, classes)
  if os.path.isdir(class_path):
    for corrent_file in os.listdir(class_path):
      file_path = os.path.join(class_path, corrent_file)
      try :
        with Image.open(file_path) as img :
          img.verify()  #verify if image can be oppened or not
      except (IOError, SyntaxError) as e:
        print(f"corrupted file : {file_path}")
        corrupted_images.append(file_path)

# No corrupted files are found using PIL

In [ ]:
#checking for corrupted files using tensorflow
bad_files =[]

for classes in os.listdir(data_path):
  class_path = os.path.join(data_path, classes)
  if os.path.isdir(class_path):
    for corrent_file in os.listdir(class_path):
      file_path = os.path.join(class_path, corrent_file)
      try:
        img_raw = tf.io.read_file(file_path)
        img_tensor = tf.io.decode_image(img_raw, channels=3, expand_animations=False)
      except Exception as e:
        print(f"TensorFlow failed to load the file:{file_path}, error: {e}")
        bad_files.append(file_path)

bad_files
# Found two corrupted files

[]

In [ ]:
#checking for corrupted files using tensorflow
bad_files_test =[]

for classes in os.listdir(test_path):
  class_path = os.path.join(test_path, classes)
  if os.path.isdir(class_path):
    for corrent_file in os.listdir(class_path):
      file_path = os.path.join(class_path, corrent_file)
      try:
        img_raw = tf.io.read_file(file_path)
        img_tensor = tf.io.decode_image(img_raw, channels=3, expand_animations=False)
      except Exception as e:
        print(f"TensorFlow failed to load the file:{file_path}, error: {e}")
        bad_files_test.append(file_path)

bad_files_test

#Found one corruoted files in the test data

[]

In [ ]:
# deleting currupted images

for corrupted_images in bad_files:
  try:
    os.remove(corrupted_images)
    print(f"{corrupted_images} is removed")
  except Exception as e:
    print(f"failed to remove {corrupted_images}, error: {e}\n")

for corrupted_images in bad_files_test:
  try:
    os.remove(corrupted_images)
    print(f"{corrupted_images} is removed")
  except Exception as e:
    print(f"failed to remove {corrupted_images}, error: {e}\n")

In [ ]:
#Loading clear data
data_dir = '/content/drive/MyDrive/NTI project/garbage-dataset'
test_dir = '/content/drive/MyDrive/NTI project/test_ds'

for classes in os.listdir(data_path):
  class_path = os.path.join(data_path, classes)
  # print(class_path)
  if os.path.isdir(class_path):
    files = os.listdir(class_path)
    print(f"{classes}: {len(files)} files") # the output shows that the paper data has lost 2 images (recheck the above cell to ensure that)

battery: 851 files
shoes: 1777 files
cardboard: 1635 files
plastic: 1784 files
trash: 897 files
glass: 2761 files
clothes: 5195 files
paper: 1518 files
biological: 897 files
metal: 920 files


In [ ]:
# Splitting data
train_ds = tf.keras.preprocessing.image_dataset_from_directory (
    data_dir,
    subset = 'training',
    batch_size = 32,
    image_size = (180,180),
    shuffle = True,
    seed = 123,
    validation_split = 0.3,
    interpolation='nearest',
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory (
    data_dir,
    subset = 'validation',
    batch_size = 32,
    image_size = (180,180),
    shuffle = False,
    seed = 123,
    validation_split = 0.3,
    interpolation='nearest'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory (
    test_dir,
    batch_size = 32,
    image_size = (180,180),
    shuffle = False,
    seed = 123,
    interpolation='nearest'
)

Found 18235 files belonging to 10 classes.
Using 12765 files for training.
Found 18235 files belonging to 10 classes.
Using 5470 files for validation.
Found 1524 files belonging to 10 classes.


In [ ]:
#visualising data
class_names = train_ds.class_names
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):  # Show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")


NameError: name 'train_ds' is not defined

In [ ]:
# preprocessing

#normalization
normalization = tf.keras.layers.Rescaling(1.0/255)

train_ds = train_ds.map(lambda x, y: (normalization(x),y))
val_ds = val_ds.map(lambda x,y:(normalization(x),y))
test_ds = test_ds.map(lambda x,y:(normalization(x),y))

#augmentaion

augmentation = tf.keras.Sequential([
    # layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
    layers.RandomFlip('Horizental'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

train_ds = train_ds.map(lambda x,y: (augmentation(x, training=True),y))

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Model construction and training

model = tf.keras.Sequential ([
    # first convlution layer
    layers.Conv2D(64, kernel_size=3),
    layers.BatchNornalization(),
    layers.activation('relu')
    layers.AveragePooling2D(pool_size=2, strides=1),


    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [ ]:
# model compilation and training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)


Epoch 1/5


In [ ]:
# Saving model
model.save('model_1_5epochs.keras')